# 0_3 A grading assistant

Teachers in general have a lot of administrations to do and one of those things is grading. Can we create a simple grade assistant to assist a Swedish teacher in grading? This exercise focuses a lot in prompt engineering and afterwards to postprocess the output using Pydantic.

#### b) Read these data into python and tell your LLM to grade them.

In [8]:
from pathlib import Path
current_folder = Path.cwd()

file_path = f"{current_folder}/assets/student_text_1.txt" 
with open(file_path, "r") as file:
    read_one_file = file.read()

print(read_one_file)

Inför allas blickar
En av de större andledningarna att man är tal rädd är förmodligen att man är
rädd för att folket som lyssnar på kommer göra narr av en. När man är ensam
på en scen eller i klass rummet så är det väldigt jobbigt när det sitter ett flertal
människor som sitter och tittar på en. Jag känner mig inte så säker när jag står och
gör en föreläsning eller en presentation. Känslan när man är framför människor
som man inte riktigt känner är ganska obehagligt, för man vill inte att dem ska
tycka illa om en, eller tycka att man är konstig på något vis. Men jag tror att det
går att bli av med talrädslan, om man börjar i en mindre grupp och gör sig vann
med att prata med människor utan att bli rädd eller känna obehag.
Peter Letmark skriver i Dagens Nyheter 2012-05-04 att det har att göra med
evolutionära förklaringar, att man är rädd för ormar eller hundar kan ju vara
en sak som sitter i ryggmärgen. Det kan bero på att man är rädd för att bli
utstöt från gruppen. Letmark berättar ä

In [19]:
current_folder = Path.cwd()
file_path = current_folder / "assets"

student_answers = []
for file in file_path.glob('student*.txt'):
    with open(file, "r") as f:
        student_reply = f.read()
    
    student_dict = {
        "answer" : student_reply
    }
    student_answers.append(student_dict)


print(student_answers)

[{'answer': 'Inför allas blickar\nAtt ha muntliga presentationer är något som uppleves skrämmande i dagens\nsamhälle för att vi lever i ett samhälle där man oftast inte blir tvingade till att\ngöra något. Muntliga presentationer är något man, i de flesta fall brukar bli\ntvungen att göra i skolan först. Det händer när man är i den åldern då man\ntänker väldigt mycket på vad ens vänner tycker om en. Så att behöva stå framför\nde och presentera något kan kännas skrämmande för då tror man att de som\ntittar på dig dömer dig. Vilket blir något som fastar med en även när man blir\näldre. Muntliga presentationer är inte något man gör i skolan lika ofta som att\nlämna in en uppgift så man blir aldrig riktigt van med det. Att bli van med att\ntala framför andra är ett bra sätt att bota talängslan. Desto mer man pratar inför\nen grupp människor ju mer inser man att det inte är så farligt och man kommer\nsäkert komma på några knep på att kunna hantera sina nerver om man börjar att\nprata inför a

In [22]:
from google import genai
from google.genai import types

for answer in student_answers:
    client = genai.Client()
    response = client.models.generate_content(
        model = "gemini-2.0-flash",
        config=types.GenerateContentConfig(
            system_instruction="You are a grading expert of this school exercise. You reply in Swedish. and you reply with a short explanation and the grade 1-5"),
        contents = f"{student_answers[0]["answer"]}",
    )
print(response.text)

Texten tar upp ett relevant och intressant ämne – rädslan för att tala inför publik. Du knyter an till en artikel i DN och diskuterar evolutionära perspektiv på talängslan. Dock är resonemanget ibland lite repetitivt och saknar djupare analys. Språket är relativt enkelt och innehåller en del grammatiska fel och stilistiska brister som påverkar läsbarheten.

**Betyg: 3**



#### c) Prompt to get an output of fields proposed_grade, motivation and improvements.
#### d) Now validate this with pydantic model

In [29]:
from pydantic import BaseModel, ValidationError
from typing import Literal

class Student_Answer(BaseModel):
    motivation : str
    proposed_grade : Literal["IG", "G", "VG"]
    improvements : str


for answer in student_answers:
    client = genai.Client()

    system_instruction = """
    Du är lärare och ska sätta betyg på en elevs svar. Svara på svenska med en förklaring (explanation).
    Inkludera vad eleven kan förbättra (improvements) och
    betygsätt mellan Icke Godkänt (IG), Godkänt (G) och Väl Godkänt (VG)") med proposed_grade
    Ditt svar får INTE innehålla något annat än JSON-objektet.
    """

    response = client.models.generate_content(
        model = "gemini-2.0-flash",
        config=types.GenerateContentConfig(
            response_mime_type="application/json", 
            response_schema=Student_Answer,
            system_instruction=system_instruction),
        contents = answer["answer"],
        )
    try:
        #print(response.text)
        grading_reply = Student_Answer.model_validate_json(response.text)
        print(f"Betyg: {grading_reply.proposed_grade}")
        print(f"Motivering: {grading_reply.motivation}")
        print(f"Förbättring: {grading_reply.improvements}")
        print("---"*20)
    except ValidationError as e:
        print(e)

Betyg: G
Motivering: Eleven visar förståelse för ämnet och kan identifiera och beskriva rädslan för att tala inför publik. Texten refererar till en relevant artikel och kopplar historiska perspektiv till nutida utmaningar. Argumentationen är dock något generell och skulle vinna på att vara mer konkret och detaljerad. Eleven behöver också utveckla sin förmåga att analysera och dra egna slutsatser baserat på informationen.
Förbättring: För att höja betyget bör eleven fokusera på att utveckla en tydligare och mer strukturerad argumentation. Det är viktigt att presentera specifika exempel och att analysera dessa på ett djupare plan. Eleven kan också dra egna slutsatser och komma med egna reflektioner kring ämnet, snarare än att enbart återge information från artikeln. Slutligen bör eleven arbeta med att förbättra sin förmåga att koppla samman olika idéer och skapa en röd tråd genom hela texten.
------------------------------------------------------------
Betyg: G
Motivering: Eleven visar f

#### e) Output a folder with the following txt files: proposed_grade.txt, motivation.txt and improvements.txt

In [40]:
class Student_Answer(BaseModel):
    motivation : str
    proposed_grade : Literal["IG", "G", "VG"]
    improvements : str

student_id = 1
for answer in student_answers:
    client = genai.Client()

    system_instruction = """
    Du är lärare och ska sätta betyg på en elevs svar. Svara på svenska med en förklaring (explanation).
    Inkludera vad eleven kan förbättra (improvements) och
    betygsätt mellan Icke Godkänt (IG), Godkänt (G) och Väl Godkänt (VG)") med proposed_grade
    Ditt svar får INTE innehålla något annat än JSON-objektet.
    """

    response = client.models.generate_content(
        model = "gemini-2.0-flash",
        config=types.GenerateContentConfig(
            response_mime_type="application/json", 
            response_schema=Student_Answer,
            system_instruction=system_instruction),
        contents = answer["answer"],
        )
    try:
        grading_reply = Student_Answer.model_validate_json(response.text)
        
        student_folder = Path.cwd() / "assets" / "grading_result" / f"student_{student_id}"
        student_folder.mkdir(parents=True, exist_ok=True)
        
        with open(student_folder / "proposed_grade.txt", mode="a") as f:
            f.write(grading_reply.proposed_grade)
        with open(student_folder / "motivation.txt", mode="a") as f:
            f.write(grading_reply.motivation)
        with open(student_folder / "improvements.txt", mode="a") as f:
            f.write(grading_reply.improvements)
        print(f"Student-{student_id} has been graded")
        print("---"*20)
        student_id +=1
    except ValidationError as e:
        print(e)

Student-1 has been graded
------------------------------------------------------------
Student-2 has been graded
------------------------------------------------------------


#### f) Go into skolverket for Svenska 1 and copy "Betygskriterier" for "Svenska 1". These are the criterias for the different grades. Paste this into a file called criterias.txt.
#### g) Now repeat b)-e) but with the criterias in your prompt as well. Can you see any differences in the outputs?
#### h) Can you improve the output quality by providing few shot examples?

In [42]:
class Student_Answer(BaseModel):
    motivation : str
    proposed_grade : Literal["IG", "G", "VG"]
    improvements : str

student_id = 1
for answer in student_answers:
    client = genai.Client()

    with open(Path.cwd()/ "assets" / "criteria.txt", "r") as f:
        system_instruction = f"""
        Du är lärare och ska sätta betyg på en elevs svar. 
        Svara på svenska.
        Utgå från denna rättningsmall: {f.read()}
        Ditt svar ska innehålla:
        - motivation (motivation).
        - Inkludera vad eleven kan förbättra (improvements)
        - Betygsätt mellan Icke Godkänt (IG), Godkänt (G) och Väl Godkänt (VG)") i proposed_grade
        Ditt svar får INTE innehålla något annat än JSON-objektet.

        I din motivation, strukturera svaret efter Innehåll, Disposition, och Språk.
        """

    response = client.models.generate_content(
        model = "gemini-2.0-flash",
        config=types.GenerateContentConfig(
            response_mime_type="application/json", 
            response_schema=Student_Answer,
            system_instruction=system_instruction),
        contents = answer["answer"],
        )
    try:
        grading_reply = Student_Answer.model_validate_json(response.text)
        
        student_folder = Path.cwd() / "assets" / "mtx_grading_result" / f"student_{student_id}"
        student_folder.mkdir(parents=True, exist_ok=True)
        
        with open(student_folder / "proposed_grade.txt", mode="a") as f:
            f.write(grading_reply.proposed_grade)
        with open(student_folder / "motivation.txt", mode="a") as f:
            f.write(grading_reply.motivation)
        with open(student_folder / "improvements.txt", mode="a") as f:
            f.write(grading_reply.improvements)
        print(f"MTX_Student-{student_id} has been graded")
        print("---"*20)
        student_id +=1
    except ValidationError as e:
        print(e)

MTX_Student-1 has been graded
------------------------------------------------------------
MTX_Student-2 has been graded
------------------------------------------------------------
